In [ ]:
import cv2
import numpy as np

def count_and_visualize_contours(
    img_path,
    thresh=50,                 # 「低一些」的 threshold
    min_area=5000,
    kernel_size=11,
    iterations=5,
    black_tol=40,              # 接近黑色的容忍度
    fill_holes=True,           # 是否用 fillPoly 先把輪廓填滿再計數
    save_path=None
):
    # 讀圖 (BGR)
    img = cv2.imread(img_path, cv2.IMREAD_COLOR)
    if img is None:
        raise ValueError(f"Failed to read image: {img_path}")

    # 0) 先把接近黑色的像素轉成白色（避免反相後成亮色誤判）
    lower_black = np.array([0, 0, 0], np.uint8)
    upper_black = np.array([black_tol, black_tol, black_tol], np.uint8)
    mask_black = cv2.inRange(img, lower_black, upper_black)
    img[mask_black > 0] = (255, 255, 255)

    # 1) 反相
    inv = 255 - img

    # 2) 灰階
    gray = cv2.cvtColor(inv, cv2.COLOR_BGR2GRAY)

    # 3) 二值化（前景=255 / 背景=0）
    _, binary = cv2.threshold(gray, thresh, 255, cv2.THRESH_BINARY)

    # 4) 膨脹
    kernel = np.ones((kernel_size, kernel_size), np.uint8)
    dilated = cv2.dilate(binary, kernel, iterations=iterations)

    # 5) 第一次輪廓（僅外輪廓）
    contours, _ = cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # 6) 填滿多邊形以避免內部空洞造成重複計數
    if fill_holes:
        filled = np.zeros_like(dilated)
        # 將所有外輪廓直接填白（實心）
        cv2.fillPoly(filled, pts=contours, color=255)

        # 用填滿後的 mask 再找一次外輪廓
        contours2, _ = cv2.findContours(filled, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        work_mask = filled
        work_contours = contours2
    else:
        work_mask = dilated
        work_contours = contours

    # 7) 依面積篩選
    large_contours = [cnt for cnt in work_contours if cv2.contourArea(cnt) > min_area]
    count = len(large_contours)

    # 8) 視覺化（畫在「原圖經黑→白處理後」的版本上）
    vis = img.copy()
    cv2.drawContours(vis, large_contours, -1, (0, 255, 0), 2)
    for i, cnt in enumerate(large_contours, 1):
        x, y, w, h = cv2.boundingRect(cnt)
        cv2.rectangle(vis, (x, y), (x+w, y+h), (0, 255, 0), 2)
        area = int(cv2.contourArea(cnt))
        cv2.putText(vis, f"#{i} area={area}", (x, y-5),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2, cv2.LINE_AA)

    cv2.putText(vis, f"Count (area>{min_area}) = {count}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1.0, (36, 255, 12), 2, cv2.LINE_AA)

    if save_path:
        cv2.imwrite(save_path, vis)

    return count, vis, binary, work_mask



img = cv2.imread('temp.png')
print(img.shape)

# ---- 範例使用 ----
count, vis, binary, dilated = count_and_visualize_contours(
    "temp.png",
    thresh=50,        # 可依影像亮度調整：越低→越「寬鬆」地抓前景
    min_area=5000,
    kernel_size=11,
    iterations=3,
    save_path="result_vis.png"
)
print("Found contours:", count)
cv2.imwrite("binary.png", binary)
cv2.imwrite("dilated.png", dilated)
cv2.imwrite("vis.png", vis)
 